In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from datetime import datetime
import pandas as pd
from IPython.display import clear_output, display
from PIL import Image
import time

# === Load Models ===
age_model = load_model("age_model.h5")  # Full saved model

emotion_model = load_model("emotion_model.h5")




# === Labels ===
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
age_labels = ['0-12', '13-18', '19-35', '36-60', '60+']

# === CSV Logger Setup ===
log_file = "log_data.csv"
log_data = []

# === Start Webcam ===
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
print("Webcam started. Running for 20 seconds...")

start_time = time.time()
duration = 20  # seconds

while time.time() - start_time < duration:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img_gray = gray[y:y+h, x:x+w]
        face_img_color = frame[y:y+h, x:x+w]

        # === Predict Age ===
        age_face = cv2.resize(face_img_color, (64, 64)) / 255.0
        age_face = np.expand_dims(age_face, axis=0)  # shape: (1, 64, 64, 3)

        age_preds = age_model.predict(age_face, verbose=0)
        age_class = np.argmax(age_preds)
        age_text = age_labels[age_class]

        # === Decision: Age Group Check ===
        if age_class == 0 or age_class == 4:
            label = f"Age: {age_text} - Not Allowed"
            color = (0, 0, 255)
            emotion_text = "N/A"
        else:
            # === Predict Emotion ===
            emo_face = cv2.resize(face_img_gray, (48, 48)) / 255.0
            emo_face = np.expand_dims(emo_face, axis=(0, -1))  # shape: (1, 48, 48, 1)
            emotion_preds = emotion_model.predict(emo_face, verbose=0)
            emotion_class = np.argmax(emotion_preds)
            emotion_text = emotion_labels[emotion_class]
            label = f"Age: {age_text} | Emotion: {emotion_text}"
            color = (0, 255, 0)

        # === Display Results ===
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # === Log Data ===
        log_data.append({
            "Time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Age Group": age_text,
            "Emotion": emotion_text
        })

    # === Display in Jupyter Notebook ===
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    clear_output(wait=True)
    display(img)

# === Cleanup ===
cap.release()
cv2.destroyAllWindows()

# === Save Log ===
df = pd.DataFrame(log_data)
df.to_csv(log_file, index=False)
print(f"\n✅ Detection completed. Log saved to '{log_file}'")
